In [ ]:

import os
import math
import numpy as np
import pandas as pd
import cv2
import joblib
from typing import Tuple
from sklearn.base import BaseEstimator


In [ ]:

def calculate_angle(x1, y1, x2, y2):
    return np.degrees(np.arctan2(y2 - y1, x2 - x1))

def analyze_image(image_path: str, crop_size: int = 200, calibration_mm_per_pixel: float = 0.00654) -> Tuple[pd.DataFrame, dict]:
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    h, w = gray.shape
    center_y, center_x = h // 2, w // 2
    zoomed_crop = gray[center_y - crop_size//2:center_y + crop_size//2,
                       center_x - crop_size//2:center_x + crop_size//2]

    edges = cv2.Canny(zoomed_crop, 50, 150)
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=30, minLineLength=10, maxLineGap=5)

    if lines is None:
        return pd.DataFrame(), {}

    measurements = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        length_px = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
        width_px = abs(y2 - y1) if abs(y2 - y1) > 0 else 1
        measurements.append({
            'length_mm': length_px * calibration_mm_per_pixel,
            'width_mm': width_px * calibration_mm_per_pixel,
            'angle_deg': calculate_angle(x1, y1, x2, y2)
        })

    return pd.DataFrame(measurements), {'image': image_path}


In [ ]:

def extract_features_from_folder(folder_path: str) -> pd.DataFrame:
    feature_list = []
    for file in os.listdir(folder_path):
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            img_path = os.path.join(folder_path, file)
            df, meta = analyze_image(img_path)
            if not df.empty:
                avg_length = df['length_mm'].mean()
                avg_width = df['width_mm'].mean()
                avg_angle = df['angle_deg'].mean()
                feature_list.append({
                    'image': file,
                    'avg_length_mm': avg_length,
                    'avg_width_mm': avg_width,
                    'avg_angle_deg': avg_angle
                })
    return pd.DataFrame(feature_list)


In [ ]:

def load_models(reg_model_path: str, clf_model_path: str) -> Tuple[BaseEstimator, BaseEstimator]:
    reg_model = joblib.load(reg_model_path)
    clf_model = joblib.load(clf_model_path)
    return reg_model, clf_model


In [ ]:

def predict(features: pd.DataFrame, reg_model: BaseEstimator, clf_model: BaseEstimator) -> pd.DataFrame:
    X = features[['avg_length_mm', 'avg_width_mm', 'avg_angle_deg']]
    features['predicted_voltage'] = reg_model.predict(X)
    features['predicted_type'] = clf_model.predict(X)
    return features


In [ ]:

def main_pipeline(image_folder: str, reg_model_path: str, clf_model_path: str) -> pd.DataFrame:
    features = extract_features_from_folder(image_folder)
    if features.empty:
        print("No valid features extracted.")
        return pd.DataFrame()
    reg_model, clf_model = load_models(reg_model_path, clf_model_path)
    results = predict(features, reg_model, clf_model)
    return results


In [ ]:

# Example usage (update paths accordingly):
# image_folder = "path/to/image/folder"
# reg_model_path = "path/to/regression_model.pkl"
# clf_model_path = "path/to/classification_model.pkl"
# output = main_pipeline(image_folder, reg_model_path, clf_model_path)
# display(output)
